## Run this notebook

You can launch this notebook using mybinder, by clicking the button below.

<a href="https://mybinder.org/v2/gh/NASA-IMPACT/GHGC_docs/HEAD?labpath=user_data_notebooks/wetland_methane_emissions.ipynb">
<img src="https://mybinder.org/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> 
</a>

## Approach

   1. Identify available dates and temporal frequency of observations for the given collection using the GHGC API `/stac` endpoint. Collection processed in this notebook is ODIAC CO2 emissions version 2022.
   2. Pass the STAC item into raster API `/stac/tilejson.json` endpoint
   3. We'll visualize two tiles (side-by-side) allowing for comparison of each of the time points using `folium.plugins.DualMap`
   4. After the visualization, we'll perform zonal statistics for a given polygon.
   

## About the Data

The EMIT instrument builds upon NASA’s long history of developing advanced imaging spectrometers for new science and applications. EMIT launched to the International Space Station (ISS) on July 14, 2022. The data shows high-confidence research grade methane plumes from point source emitters - updated as they are identified - in keeping with JPL Open Science and Open Data policy.

# Installing the required libraries.
Please run the next cell to install all the required libraries to run the notebook.

## Querying the STAC API

In [130]:
import requests
from folium import Map, TileLayer
from pystac_client import Client

In [131]:
# Provide STAC and RASTER API endpoints
STAC_API_URL = "http://dev.ghg.center/api/stac"
RASTER_API_URL = "https://dev.ghg.center/api/raster"

#Please use the collection name similar to the one used in STAC collection.
# Name of the collection for wetland methane monthly emissions. 
collection_name = "nasa-jpl-plumes-emissions-updated"

In [132]:
# Fetching the collection from STAC collections using appropriate endpoint.
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()
collection

{'id': 'nasa-jpl-plumes-emissions-updated',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/collections/nasa-jpl-plumes-emissions-updated/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/collections/nasa-jpl-plumes-emissions-updated'}],
 'title': 'CH4 plumes emissions',
 'assets': None,
 'extent': {'spatial': {'bbox': [[-121.71466779194687,
     -38.78784262561637,
     151.09064663683435,
     50.24673720047911]]},
  'temporal': {'interval': [['2022-08-10 00:00:00+00',
     '2023-05-04 00:00:00+00']]}},
 'license': 'CC0-1.0',
 'keywor

Examining the contents of our `collection` under `summaries` we see that the data is available from January 2000 to December 2021. By looking at the `dashboard:time density` we observe that the periodic frequency of these observations is monthly. 

In [133]:
# Check total number of items available
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit=500").json()["features"]
print(f"Found {len(items)} items")

Found 401 items


In [134]:
items[0]

{'id': 'emit20230504t135454_o12409_s000_l1b_ch4mf_b0106_v01_mv0_p2_20230504',
 'bbox': [-46.450625050282454,
  -23.700764982003268,
  -46.41212654134425,
  -23.630816986890196],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/collections/nasa-jpl-plumes-emissions-updated'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/collections/nasa-jpl-plumes-emissions-updated'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/collections/nasa-jpl-plumes-emissions-updated/items/emit20230504t135454_o12409_s000_l1b_ch4mf_b0106_v01_mv0_p2_20230504'}],
 'assets': {'ch4-plume-emissions': {'href': 's3://ghgc-

This makes sense as there are 22 years between 2000 - 2021, with 12 months per year, meaning 264 records in total.  

Below, we are entering the minimum and maximum values to provide our upper and lower bounds in `rescale_values`.

## Exploring Changes in Methane emissions (CH4) levels using the Raster API

We will explore changes in methane emissions in wetland regions. In this notebook, we'll explore the impacts of these emissions and explore these changes over time. We'll then visualize the outputs on a map using `folium`. 

In [135]:
# to access the year value from each item more easily, this will let us query more explicity by year and month (e.g., 2020-02)
items = {item["id"]: item for item in items} 

In [137]:
rescale_values = {"max":items[list(items.keys())[0]]["assets"]["ch4-plume-emissions"]["raster:bands"][0]["histogram"]["max"], "min":items[list(items.keys())[0]]["assets"]["ch4-plume-emissions"]["raster:bands"][0]["histogram"]["min"]}

Now we will pass the item id, collection name, and `rescaling_factor` to the `Raster API` endpoint. We will do this twice, once for December 2001 and again for December 2021, so that we can visualize each event independently. 

In [138]:
color_map = "magma" # please select the color ramp from matplotlib library.
december_2001_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items['emit20230504t135454_o12409_s000_l1b_ch4mf_b0106_v01_mv0_p2_20230504']['collection']}&item={items['emit20230504t135454_o12409_s000_l1b_ch4mf_b0106_v01_mv0_p2_20230504']['id']}"
    "&assets=ch4-plume-emissions"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
december_2001_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://2qncpyg24c.execute-api.us-west-2.amazonaws.com/api/raster/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=nasa-jpl-plumes-emissions-updated&item=emit20230504t135454_o12409_s000_l1b_ch4mf_b0106_v01_mv0_p2_20230504&assets=ch4-plume-emissions&color_formula=gamma+r+1.05&colormap_name=magma&rescale=-2927.2822265625%2C5396.69921875'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-46.450625050282454,
  -23.700764982003268,
  -46.41212654134425,
  -23.630816986890196],
 'center': [-46.43137579581335, -23.66579098444673, 0]}

In [139]:
august_11_2022 = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items['emit20230504t135454_o12409_s000_l1b_ch4mf_b0106_v01_mv0_p2_20230504']['collection']}&item={items['emit20230504t135454_o12409_s000_l1b_ch4mf_b0106_v01_mv0_p2_20230504']['id']}"
    "&assets=ch4-plume-emissions"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
).json()
august_11_2022

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://2qncpyg24c.execute-api.us-west-2.amazonaws.com/api/raster/stac/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=nasa-jpl-plumes-emissions-updated&item=emit20230504t135454_o12409_s000_l1b_ch4mf_b0106_v01_mv0_p2_20230504&assets=ch4-plume-emissions&color_formula=gamma+r+1.05&colormap_name=magma&rescale=-2927.2822265625%2C5396.69921875'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-46.450625050282454,
  -23.700764982003268,
  -46.41212654134425,
  -23.630816986890196],
 'center': [-46.43137579581335, -23.66579098444673, 0]}

## Visualizing CH4 emissions


In [144]:
# We'll import folium to map and folium.plugins to allow mapping side-by-side
import folium
import folium.plugins

# Set initial zoom and center of map for CO2 Layer
map_ = folium.Map(location=(-23.66, -46.43), zoom_start=13)

# December 2001
map_layer_2001 = TileLayer(
    tiles=august_11_2022["tiles"][0],
    attr="GHG",
    opacity=1,
)
map_layer_2001.add_to(map_)

# visualising the map
map_



# Calculating the zonal statistics

## 

In [159]:
# Texas, Dallas, USA AOI
texas_dallas_aoi = {
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates":[
          [
[
          -46.450625050282454,
          -23.700764982003268
        ],
        [
          -46.41212654134425,
          -23.700764982003268
        ],
        [
          -46.41212654134425,
          -23.630816986890196
        ],
        [
          -46.450625050282454,
          -23.630816986890196
        ],
        [
          -46.450625050282454,
          -23.700764982003268
        ]
        ]],
        "type": "Polygon",
    },
}

In [160]:
# We'll plug in the coordinates for a location
# central to the study area and a reasonable zoom level

import folium

aoi_map = Map(
    tiles="OpenStreetMap",
    location=[
        -23.630816986890196,
          -46.450625050282454
    ],
    zoom_start=6,
)

folium.GeoJson(texas_dallas_aoi, name="Texas, Dallas").add_to(aoi_map)
aoi_map

In [40]:
# Check total number of items available
items = requests.get(
    f"{STAC_API_URL}/collections/{collection_name}/items?limit=600"
).json()["features"]
print(f"Found {len(items)} items")

Found 504 items


In [41]:
# Explore one item to see what it contains
items[0]

{'id': 'nasa-gsfc-ch4-wetlands-emissions-202112',
 'bbox': [-180.0, -90.0, 180.0, 90.0],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/collections/nasa-gsfc-ch4-wetlands-emissions'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/collections/nasa-gsfc-ch4-wetlands-emissions'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://gkynb1qvnl.execute-api.us-west-2.amazonaws.com/api/stac/collections/nasa-gsfc-ch4-wetlands-emissions/items/nasa-gsfc-ch4-wetlands-emissions-202112'}],
 'assets': {'ch4-wetlands-emissions': {'href': 's3://ghgc-data-store-dev/NASA_GSFC_ch4_wetlands_monthly/NASA_GSFC_ch4_wl_ch4_wetlands_v22_x720_y360_t12_202112.tif',
   'type':

In [42]:
# the bounding box should be passed to the geojson param as a geojson Feature or FeatureCollection
def generate_stats(item, geojson):
    result = requests.post(
        f"{RASTER_API_URL}/cog/statistics",
        params={"url": item["assets"]["ch4-wetlands-emissions"]["href"]},
        json=geojson,
    ).json()
    print(result)
    return {
        **result["properties"],
        "start_datetime": item["properties"]["start_datetime"],
    }

In [43]:
for item in items:
    print(item["properties"]["start_datetime"])
    break

2021-12-01T00:00:00+00:00


With the function above we can generate the statistics for the AOI.

In [44]:
%%time
stats = [generate_stats(item, texas_dallas_aoi) for item in items]

{'detail': [{'loc': ['body', 'type'], 'msg': "unexpected value; permitted: 'FeatureCollection'", 'type': 'value_error.const', 'ctx': {'given': 'Feature', 'permitted': ['FeatureCollection']}}, {'loc': ['body', 'features'], 'msg': 'field required', 'type': 'value_error.missing'}, {'loc': ['body', 'geometry', 'coordinates'], 'msg': 'wrong tuple length 1, expected 2', 'type': 'value_error.tuple.length', 'ctx': {'actual_length': 1, 'expected_length': 2}}, {'loc': ['body', 'geometry', 'coordinates'], 'msg': 'wrong tuple length 1, expected 3', 'type': 'value_error.tuple.length', 'ctx': {'actual_length': 1, 'expected_length': 3}}, {'loc': ['body', 'geometry', 'type'], 'msg': "unexpected value; permitted: 'Point'", 'type': 'value_error.const', 'ctx': {'given': 'Polygon', 'permitted': ['Point']}}, {'loc': ['body', 'geometry', 'coordinates', 0], 'msg': 'wrong tuple length 4, expected 2', 'type': 'value_error.tuple.length', 'ctx': {'actual_length': 4, 'expected_length': 2}}, {'loc': ['body', 'geom

KeyError: 'properties'

In [ ]:
stats[0]

In [ ]:
import pandas as pd


def clean_stats(stats_json) -> pd.DataFrame:
    df = pd.json_normalize(stats_json)
    df.columns = [col.replace("statistics.b1.", "") for col in df.columns]
    df["date"] = pd.to_datetime(df["start_datetime"])
    return df


df = clean_stats(stats)
df.head(5)

## Visualizing the Data as a Time Series
We can now explore the ODIAC fossil fuel emission time series available (January 2000 -December 2021) for the Texas, Dallas area of USA. We can plot the data set using the code below:

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20, 10))


plt.plot(
    df["date"],
    df["max"],
    color="red",
    linestyle="-",
    linewidth=0.5,
    label="Max monthly CH4 emissions",
)

plt.legend()
plt.xlabel("Years")
plt.ylabel("CH4 emissions g/m2")
plt.title("CH4 emission Values for Texas, Dallas (2000-2021)")

In [ ]:
print(items[2]["properties"]["start_datetime"])

In [ ]:
october_tile = requests.get(
    f"{RASTER_API_URL}/stac/tilejson.json?collection={items[2]['collection']}&item={items[2]['id']}"
    "&assets=ch4-wetlands-emissions"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}",
).json()
october_tile

In [ ]:
# Use bbox initial zoom and map
# Set up a map located w/in event bounds
import folium

aoi_map_bbox = Map(
    tiles="OpenStreetMap",
    location=[
        -22.421460,
        14.268801,
    ],
    zoom_start=8,
)

map_layer = TileLayer(
    tiles=october_tile["tiles"][0],
    attr="GHG", opacity = 0.5
)

map_layer.add_to(aoi_map_bbox)

aoi_map_bbox

## Summary

In this case study we have successfully visualized the CH4 emissions from wetland.